In [134]:
from plyfile import PlyData, PlyElement
import numpy as np

In [135]:
plydata = PlyData.read('tex.ply')

In [136]:
len(plydata)

2

In [137]:
print(plydata)

ply
format ascii 1.0
comment zipper output
element vertex 4
property float x
property float y
property float z
element face 4
property list uchar int vertex_indices
end_header


In [138]:
plydata['vertex']

PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float')), count=4, comments=[])

In [144]:
plydata['vertex'][1]

(0., 1., 0.)

In [140]:
plydata['face'][0][0]

array([0, 1, 2], dtype=int32)

In [141]:
count = 0
for x in plydata['vertex']:
#     print(x[0], x[1], x[2])
#     print(str([x[0], x[1], x[2]]))
    count += 1

In [142]:
count

4

In [143]:
#calculate and store area-weighted vertex normals for future use#
vertex_normals = np.zeros((count, 3))
for x in plydata['face']:
    #get indexs of vertices in the face #
    index = x[0]
    
    #get corresponding vertices# 
    point_a = plydata['vertex'][index[0]]
    point_b = plydata['vertex'][index[1]]
    point_c = plydata['vertex'][index[2]]
    
    #use np to reconstruct to eliminate extra properties rather than xyz and vector calculation#
    t_point_a = np.array([point_a[0], point_a[1], point_a[2]])
    t_point_b = np.array([point_b[0], point_b[1], point_b[2]])
    t_point_c = np.array([point_c[0], point_c[1], point_c[2]])
    
    #construct vectors in the face#
    v1 = t_point_b - t_point_a
    v2 = t_point_c - t_point_a
    
    #calculate normal and then normalize#
    normal = np.cross(v1, v2)
    normal = normal / (np.linalg.norm(normal))
    print(normal)
    
    #add normals to relavant vertex indexs 
    vertex_normals[index[0]] += normal
    vertex_normals[index[1]] += normal
    vertex_normals[index[2]] += normal

[0. 0. 1.]
[ 0.57735026 -0.57735026  0.57735026]
[0.57735026 0.57735026 0.57735026]
[1. 0. 0.]


In [131]:
print(vertex_normals)

[[ 1.15470052  0.          2.15470052]
 [ 1.57735026  0.57735026  1.57735026]
 [ 1.57735026 -0.57735026  1.57735026]
 [ 2.15470052  0.          1.15470052]]


In [132]:
normalized_vector_normals = []
for x in vertex_normals:
    if (np.linalg.norm(x) > 0.0):
        x = x / (np.linalg.norm(x))
    normalized_vector_normals.append(x)

In [133]:
print(normalized_vector_normals)

[array([0.47234749, 0.        , 0.88141242]), array([0.68455032, 0.2505628 , 0.68455032]), array([ 0.68455032, -0.2505628 ,  0.68455032]), array([0.88141242, 0.        , 0.47234749])]


In [57]:
f = open("lucy.xyz", "w")

In [58]:
f.write(str(count) + "\n")

9

In [59]:
for x in plydata['vertex']:
    f.write(str(x[0]) + " ")
    f.write(str(x[1]) + " ")
    f.write(str(x[2]) + "\n")